# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def exp_return(data, P):
    
    # Initialize the sum of weighted prices
    weighted_sum = 0
    
    # Iterate over each row of the DataFrame
    for index, row in data.iterrows():
        # Multiply the price by the probability and add it to the weighted sum
        weighted_sum += row['Price in 1 Year'] * row['Probability']
    
    # Calculate the expected return by subtracting the current price and dividing by the current price
    expected_return = (weighted_sum - P) / P
    
    return expected_return

Write the function ```volatility```, which calculates the volatilityof the stock, given the current stock price ```P```.

In [4]:
def volatility(data, P):
    
    # Calculate the expected return
    expected_return = exp_return(data, P)

    # Calculate the weighted sum of squared deviations
    weighted_squared_deviations = sum(data['Probability'] * ((data['Price in 1 Year'] / P - 1 - expected_return) ** 2))

    # The volatility is the square root of the weighted sum
    volatility = weighted_squared_deviations ** 0.5
    
    return volatility

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [5]:
def interval(data, P):
    
    expected_return = exp_return(data, P)
    vol = volatility(data, P)
    Z = 1.96

    # Calculate standard error
    standard_error = Z * vol /np.sqrt(len(data))
    
    # Lower and upper bounds
    lower_bound = expected_return - standard_error
    upper_bound = expected_return + standard_error

    return [lower_bound, upper_bound]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [6]:
def exp_return_capm(beta, market_return, rf):
    
    expected_return = rf + beta * (market_return - rf)
    
    return expected_return

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [7]:
def clean_data(path):
    
     # Load the data
    df = pd.read_csv(path)
    
    # Convert RET to numeric, forcing non-numeric to NaN
    df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
    
    # Drop rows where RET is NaN
    df = df.dropna(subset=['RET'])
    
    # Count the number of observations for each PERMNO and filter out those with < 500 observations
    counts = df['PERMNO'].value_counts()
    df = df[df['PERMNO'].isin(counts[counts >= 500].index)]
    
    # Ensure only the required columns are present
    df = df[['PERMNO', 'date', 'CUSIP', 'RET']]
    
    return df
# This should yield data with 403316 rows × 4 columns
print(clean_data("monthlycrsp_mod6.csv"))

         PERMNO        date     CUSIP       RET
9027      10065  1970-01-30  00621210 -0.144375
9028      10065  1970-02-27  00621210  0.105882
9029      10065  1970-03-31  00621210  0.017857
9030      10065  1970-04-30  00621210 -0.078947
9031      10065  1970-05-29  00621210 -0.020952
...         ...         ...       ...       ...
3393558   84129  2022-08-31  98970110  0.016315
3393559   84129  2022-09-30  98970110 -0.075777
3393560   84129  2022-10-31  98970110  0.021235
3393561   84129  2022-11-30  98970110  0.005583
3393562   84129  2022-12-30  98970110 -0.051332

[403316 rows x 4 columns]


Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [8]:
def avg_return(data):
    
    # Load data from csv file
    data = clean_data(data)
    
    # Group by 'CUSIP' and calculate the mean of 'RET' for each group
    avg_return_data = data.groupby('CUSIP')['RET'].mean().reset_index()
    
    # Create the output DataFrame with the correct column names
    output = pd.DataFrame(data={
        "CUSIP": avg_return_data['CUSIP'],
        "Average Return": avg_return_data['RET']
    })
    
    return output

In [9]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    
    # Load data from csv file
    data = clean_data(data)
    
    # Group by 'CUSIP' and calculate the standard deviation of 'RET' for each group
    volatility_data = data.groupby('CUSIP')['RET'].std().reset_index()
    
    # Rename the columns according to the requirement
    volatility_data.rename(columns={'RET': 'Volatility'}, inplace=True)
    
    # Create the output DataFrame with the correct column names
    output = pd.DataFrame(data={
        "CUSIP": volatility_data['CUSIP'],
        "Volatility": volatility_data['Volatility']
    })
    
    return output

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [10]:
def correlation(ret_data, vol_data):
    
    # Merge the two DataFrames on 'CUSIP'
    merged_data = pd.merge(ret_data, vol_data, on='CUSIP')
    # Calculate the correlation between 'Average Return' and 'Volatility'
    
    corr = merged_data['Average Return'].corr(merged_data['Volatility'])
    
    return corr